Job Shop Scheduling is a complex optimization problem that involves assigning jobs to machines while minimizing the total completion time (makespan). In this Colab, we implement a solution using Integer Linear Programming (ILP) with the Gurobi solver, modeling sequencing and non-overlapping constraints to achieve optimal scheduling, based on:

**King, B., & Hildebrand, R. (2024). Job Shop Scheduling with Integer Programming, Shifting Bottleneck, and Decision Diagrams: A Computational Study. arXiv.** https://arxiv.org/abs/2407.18111

In [ ]:
pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 33.1 MB/s eta 0:00:00


In [ ]:
#from gurobipy import *
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np
import itertools
import random
import math

## 1. Gurobi solving the problem:

Problem Inputs

In [ ]:
# Problem parameters
jobs = ["T1", "T2", "T3"]
machines = ["M1", "M2", "M3", "M4"]

# Processing times for each job on each machine
processing_time = {
    "T1": [30, 25, 40, 20],
    "T2": [25, 25, 15, 10],
    "T3": [20, 10, 30, 10],
}

Guroby inputs

In [ ]:
# Big M parameter
M = 1000

# Model
m = Model("Ejemplo")

# >> Start time variables <<
# S_j,m = Start time of job j on machine m
start = {}
for job in jobs:
    for i, machine in enumerate(machines):
        start[job, machine] = m.addVar(vtype=GRB.CONTINUOUS, name=f"S_{job}_{machine}")

# Makespan variable
Cmax = m.addVar(vtype=GRB.CONTINUOUS, name="Cmax")

# Binary order variables (between each pair of jobs on each machine)
# X_j1_j2,m = Binary variable indicating if job j1 precedes job j2 on machine m
order = {}
for m_index, machine in enumerate(machines):
    for j1 in range(len(jobs)):
        for j2 in range(j1 + 1, len(jobs)):
            order[jobs[j1], jobs[j2], machine] = m.addVar(vtype=GRB.BINARY, name=f"X_{jobs[j1]}_{jobs[j2]}_{machine}")

# >> Sequence constraints (Flow Shop) <<
# S_j,m+1 >= S_j,m + p_j,m
# p_j,m = Processing time of job j on machine m
for job in jobs:
    for i in range(len(machines) - 1):
        m.addConstr(
            start[job, machines[i + 1]] >= start[job, machines[i]] + processing_time[job][i],
            f"Seq_{job}_{machines[i]}_{machines[i+1]}"
        )

# >> No-overlap constraints on machines <<
for m_index, machine in enumerate(machines):
    for j1 in range(len(jobs)):
        for j2 in range(j1 + 1, len(jobs)):
            job1, job2 = jobs[j1], jobs[j2]
            p1, p2 = processing_time[job1][m_index], processing_time[job2][m_index]
            # Job1 before Job2
            # S_j2,m >= S_j1,m + p_j1,m - M * X_j1_j2,m
            m.addConstr(
                start[job2, machine] >= start[job1, machine] + p1 - M * order[job1, job2, machine],
                f"NoOverlap_{job1}_{job2}_{machine}_A"
            )
            # Job2 before Job1
            # S_j1,m >= S_j2,m + p_j2,m - M * (1 - X_j1_j2,m)
            m.addConstr(
                start[job1, machine] >= start[job2, machine] + p2 - M * (1 - order[job1, job2, machine]),
                f"NoOverlap_{job2}_{job1}_{machine}_B"
            )

# >> Makespan constraints <<
# C_max >= S_j,m_end + p_j,m_end
for job in jobs:
    m.addConstr(Cmax >= start[job, machines[-1]] + processing_time[job][-1], f"Makespan_{job}")

# >> Objective function <<
m.setObjective(Cmax, GRB.MINIMIZE)

# Optimization
m.optimize()


# Time measurement
start_time = time.time()
end_time = time.time()
execution_time = end_time - start_time

NameError: name 'Model' is not defined

In [ ]:
# >> Results <<
if m.status == GRB.OPTIMAL:
    print(f"\nMinimum makespan: {Cmax.X}\n")
    for job in jobs:
        for i, machine in enumerate(machines):
            print(f"Start time of {job} on {machine}: {start[job, machine].X:.2f}")
else:
    print("No optimal solution found.")

print(f'\nExecution Time: {execution_time:.9f}')

In [ ]:
# Store results in a list of dictionaries
gantt_data = []

if m.status == GRB.OPTIMAL:
    print(f"\nMinimum makespan: {Cmax.X}\n")
    for job in jobs:
        for i, machine in enumerate(machines):
            start_time = start[job, machine].X
            duration = processing_time[job][i]
            gantt_data.append({"Job": job, "Machine": machine, "Start": start_time, "Duration": duration})

# Convert the data to a DataFrame
df = pd.DataFrame(gantt_data)

# Plot the Gantt chart
fig, ax = plt.subplots(figsize=(12, 6))

colors = ["#ff0000", "#00ff00", "#1111ff"]  # Different colors for each job
for i, job in enumerate(jobs):
    job_data = df[df["Job"] == job]
    # Add the bar with the label only once per job
    ax.barh(job_data["Machine"], job_data["Duration"], left=job_data["Start"], height=0.4,
            label=job, color=colors[i])

# Add legend outside the plot
plt.legend(title="Jobs", bbox_to_anchor=(1.05, 1), loc="upper left")

# Add labels and title
ax.set_xlabel("Time")
ax.set_ylabel("Machine")
ax.set_title("Gantt Chart - Job Scheduling")

# Configure grid
ax.grid(True, axis="x", linestyle="--", alpha=0.6)  # Vertical grid lines
ax.set_xticks(range(0, int(Cmax.X) + 5, 5))  # Grid every 5 units

# Horizontal lines (top and bottom of each bar)
y_positions = range(len(machines))
for y in y_positions:
    ax.axhline(y + 0.2, color="gray", linestyle="--", alpha=0.6)  # Top line
    ax.axhline(y - 0.2, color="gray", linestyle="--", alpha=0.6)  # Bottom line

# Show the plot
plt.tight_layout()
plt.show()


## 2. Direct Evaluation

#### 2.1 Makespan and brute force approach

In [ ]:
def cal_makespan(proc_time, job_sequence):
    jobs = list(proc_time.keys())
    num_jobs = len(jobs)
    num_machines = len(next(iter(proc_time.values())))

    comp_time = np.zeros((num_jobs, num_machines))

    # Fill completion times
    for j, job in enumerate(job_sequence):
        for m in range(num_machines):
            if j == 0 and m == 0:
                # First job and machine time
                comp_time[j, m] = proc_time[job][m]
            elif j == 0:
                # Subsequent machine times for the first job
                comp_time[j, m] = comp_time[j, m-1] + proc_time[job][m]
            elif m == 0:
                # First machine time for the subsequent jobs
                comp_time[j, m] = comp_time[j-1, m] + proc_time[job][m]
            else:
                # Subsequent machine times for the first job
                comp_time[j, m] = max(comp_time[j-1, m], comp_time[j, m-1]) + proc_time[job][m]

    return comp_time[-1, -1]

Brute force function

In [ ]:
def brute_force(proc_time, jobs, verbose=False):
    min = np.inf
    best = []
    for order in list(itertools.permutations(jobs)):
        makespan = cal_makespan(proc_time, order)
        if(makespan < min):
            min = makespan
            best = order

    if(verbose):
        print(f'Final Brute Force results')
        print(f'Min: {min} with {best}\n')
    return min, best

### 2.2 Genetic algorithm

#### Required functions

Initialize and calculate fitness variable

In [ ]:
def IniPopGen(size, num_jobs):
    # Initial Population Generation
    base = list(range(1, num_jobs + 1))
    pop = []

    for _ in range(size):
        sampled = random.sample(base, num_jobs) # Random sort
        jobs = ["T" + str(i) for i in sampled] # Convert job indices to job names
        pop.append(jobs)
    return pop

def FitCalc(pop, proc_time):
    # Population fitness calculation
    return [1 / cal_makespan(proc_time, indv) for indv in pop]

Selection of best individuals

In [ ]:
def SelBest(pop, fitness, method="rank", vs_size=2):
    # Selects best individuals using the specified method: 'rank' or 'vs'.
    # Returns the new population

    NPop = len(pop)

    if method == "rank":
        # Link population and fitness values
        rank_zip = list(zip(pop, fitness))
        # Ascending order of fitness (x[1])
        rank_zip.sort(key=lambda x: x[1])
        # Probability ranks (Highest to the best)
        ranks = np.arange(NPop, 0, -1)
        probs = ranks / ranks.sum()

        # Select individuals according to "probs"
        sel_idx = np.random.choice(
            range(NPop),
            size=NPop,
            replace=True,
            p=probs
        )

        best = [pop[i] for i in sel_idx]
        return list(best)

    elif method == "vs":
        # Tournament Selection
        selected = []
        for _ in range(NPop):
            # Random pairings
            fighters = random.sample(range(NPop), vs_size)
            # Compare fitness to choose the better individual
            best_idx = max(fighters, key=lambda i: fitness[i])
            selected.append(pop[best_idx])
        return selected

    else:
        raise ValueError("Check your selection method, it must be 'rank' or 'vs'")

Cross and mutation

In [ ]:
def CrossOX1(parent1, parent2):
    # Order Crossover (OX1)

    # Child size equal to parent size
    size = len(parent1)
    child = [None] * size

    # Select a random section of parent1
    start, end = sorted(random.sample(range(size), 2))
    child[start:end] = parent1[start:end]

    # Fill the missing parts with parent2 "genes"
    gen_pos = end # Start at the end of parent1 section
    for gene in parent2:
        if gene not in child:
            # Move gen_pos to None positions
            while child[gen_pos] is not None:
                gen_pos += 1
                # Restart the search loop (circular index)
                if gen_pos >= size:
                    gen_pos = 0
            # Assign child gene
            child[gen_pos] = gene

    return child

def Mutate(indv):
    # Swap mutation
    i, j = random.sample(range(len(indv)), 2) # Random Index pair
    indv[i], indv[j] = indv[j], indv[i] # Swap
    return indv

Memetic functions

In [ ]:
def Local_Search(indv):
# Function by Rafael Rojas
    # do-while condition
    t = -1
    while t < 0:
        # Makespan calculation for initial individual
        fitness1 = cal_makespan(processing_time, indv)
        # Makespan calculation for new individual
        indv_new = Mutate(indv)
        fitness2 = cal_makespan(processing_time, indv)
        # Makespan comparison
        t = fitness2 - fitness1
        # Change individual if a improvement is observed
        if t<0:
            indv = indv_new
    return indv


Simulated Annealing

In [ ]:
def Sim_Annealing(S):
# Function by Rafael Rojas
    # Initial solution generation
    best_makespan = cal_makespan(processing_time, S)
    best_sol = S

    # Parameters
    T0 = 100
    alfa = 0.9
    L = 100
    Tf = 1
    beta = 1/1000 # (Boltzmann)

    # Initial temperature
    T = T0

    # SA process
    while T > Tf:
        for z in range(L):
            # Generate random neighbor
            S_new = Mutate(S)
            # Calculate new and old makespans
            makespan_old = cal_makespan(processing_time, S)
            makespan_new = cal_makespan(processing_time, S_new)
            # Makespan difference
            deltaE = makespan_new - makespan_old
            # Compare makespan
            if makespan_new < best_makespan:
              # If it is better than the best, both best makespan and solution are updated
              best_makespan = makespan_new
              best_sol = S_new
            # If a improvement is observed, the solution is saved
            if deltaE < 0:
              S = S_new
            # If not, the save happens according to a declining propability
            elif math.exp(-deltaE/T) > random.random():
              S = S_new

        # Temperature update
        T = alfa*T

    return best_sol

Evolutive generation

In [ ]:
def EvolGen(pop, proc_time, sel_method="rank", vs_size=2, cross_rate=0.8, mut_rate=0.02):
    # >> 0. Local search (by Rafael Rojas)
    #for i in range(10):
      #r = random.randint(0,len(pop)-1)
      #pop[r] = Sim_Annealing(pop[r])

    # >> 1. Fitness calculation
    fitness = FitCalc(pop, proc_time)

    # Save best
    best_fit = max(fitness)
    best_indv = pop[fitness.index(best_fit)]

    # >> 2. Selection
    selected = SelBest(pop, fitness, method=sel_method, vs_size=vs_size)

    # >> 3. Crossover
    new_pop = []
    for i in range(0, len(selected), 2):
        # Select i-th parents couple
        parent1 = selected[i]
        # Check i+1-th sample
        if i + 1 < len(selected):
            parent2 = selected[i + 1]
        else:
            parent2 = selected[0]  # In case of odd population size

        # Random cross activation
        if random.random() < cross_rate:
            # A pair of childs
            child1 = CrossOX1(parent1, parent2)
            child2 = CrossOX1(parent2, parent1)
        else:
            child1, child2 = parent1[:], parent2[:]

        # Extend new population
        new_pop.extend([child1, child2])

    # >> 4. Mutation
    for i in range(len(new_pop)):
        if random.random() < mut_rate:
            new_pop[i] = Mutate(new_pop[i])

    return new_pop[:len(pop)], best_fit, best_indv


#### Testing

In [ ]:
# Example, 5 Jobs, 4 Machines
jobs = ["T1", "T2", "T3", "T4", "T5", "T6"]
machines = ["M1", "M2", "M3", "M4"]

# Processing time of each job on each machine
processing_time = {
    "T1": [30, 25, 40, 20],
    "T2": [25, 25, 15, 10],
    "T3": [20, 10, 30, 10],
    "T4": [35, 20, 25, 15],
    "T5": [15, 30, 20, 25],
    "T6": [25, 20, 30, 15],
}

#Brute force
min, best = brute_force(processing_time,jobs,verbose=True)

Final Brute Force results
Min: 205.0 with ('T3', 'T5', 'T6', 'T1', 'T4', 'T2')



In [ ]:
# Initial population generation
population = IniPopGen(20, len(jobs))

for generation in range(50):
    population, best_fit, best_indv = EvolGen(population, processing_time, sel_method="vs", vs_size=3, cross_rate=0.9, mut_rate=0.2)

print(f'Best Makespan: {1/best_fit} with {best_indv}')

Best Makespan: 209.99999999999997 with ['T3', 'T5', 'T1', 'T4', 'T6', 'T2']


In [ ]:
processing_time = {
    "T1": [54, 79, 16, 66, 58],
    "T2": [83, 3, 89, 58, 56],
    "T3": [15, 11, 49, 31, 20],
    "T4": [71, 99, 15, 68, 85],
    "T5": [77, 56, 89, 78, 53],
    "T6": [36, 70, 45, 91, 35],
    "T7": [53, 99, 60, 13, 53],
    "T8": [38, 60, 23, 59, 41],
    "T9": [27, 5, 57, 49, 69],
    "T10": [87, 56, 64, 85, 13],
    "T11": [76, 3, 7, 85, 86],
    "T12": [91, 61, 1, 9, 72],
    "T13": [14, 73, 63, 39, 8],
    "T14": [29, 75, 41, 41, 49],
    "T15": [12, 47, 63, 56, 47],
    "T16": [77, 14, 47, 40, 87],
    "T17": [32, 21, 26, 54, 58],
    "T18": [87, 86, 75, 77, 18],
    "T19": [68, 5, 77, 51, 68],
    "T20": [94, 77, 40, 31, 28],
}

jobs = ["T" + str(i) for i in range(1, 21)]
machines = ["M" + str(i) for i in range(1, 5)]

# Initial population generation
population = IniPopGen(200, len(jobs))

for generation in range(50):
    population, best_fit, best_indv = EvolGen(population, processing_time, sel_method="vs", vs_size=3, cross_rate=0.9, mut_rate=0.2)

print(f'Best Makespan: {1/best_fit} with {best_indv}')
print(f'Gap: {(1/best_fit-1278)/1278*100}%')

Best Makespan: 1330.0 with ['T15', 'T6', 'T12', 'T17', 'T19', 'T14', 'T9', 'T5', 'T10', 'T4', 'T8', 'T3', 'T11', 'T7', 'T2', 'T1', 'T16', 'T13', 'T18', 'T20']
Gap: 4.068857589984351%


### 2.3 Monte-Carlo

Random Matrix generator

In [ ]:
def samp_gen(std_dict, machines, processing_time, verbose=False):
    # Create a new dictionary with random processing times
    proc_time_MC = {}

    for job in processing_time:
        orig_times = processing_time[job]

        # Get job-specific std; fallback to 1.0 if not found
        job_std = std_dict.get(job)

        # Generate random times per machine
        rand_times = np.random.normal(loc=orig_times, scale=job_std)
        proc_time_MC[job] = list(rand_times)

        if verbose:
            print(f'| {job} | {[f"{val:.6g}" for val in proc_time_MC[job]]}')


    return proc_time_MC


Monte-carlo loop:

In [ ]:
def Monte_Carlo(NSamples, data_std, machines, processing_time, verbose=False):
    Min_List = []

    for i in range(NSamples):
        # Generate one sample
        proc_time_MC = samp_gen(data_std, machines, processing_time)

        # Evaluate the makespan using brute force
        min_val, _ = brute_force(proc_time_MC, jobs)
        Min_List.append(min_val)

    avg = np.mean(Min_List)
    std = np.std(Min_List)

    if verbose:
        print(f'Final MC results using: {NSamples} samples')
        print(f'Average: {avg}')
        print(f'Std: {std}\n')

    return avg, std


Testing

In [ ]:
# Problem parameters
jobs = ["T1", "T2", "T3"]
machines = ["M1", "M2", "M3", "M4"]

# Processing times for each job on each machine
processing_time = {
    "T1": [30, 25, 40, 20],
    "T2": [25, 25, 15, 10],
    "T3": [20, 10, 30, 10],
}

# Typical results
min, best = brute_force(processing_time, jobs, verbose=True)

Final Brute Force results
Min: 145.0 with ('T3', 'T1', 'T2')



In [ ]:
std_dict = {
    "T1": [4, 4, 4, 4],
    "T2": [4, 4, 4, 4],
    "T3": [5, 5, 5, 5]
}

# Sample Generator
new_times = samp_gen(std_dict, machines, processing_time, verbose=True)

# Monte-Carlo
avg_MC, std_MC = Monte_Carlo(1024, std_dict, machines, processing_time, verbose=True)

| T1 | ['33.1526', '31.3851', '45.1529', '14.0319']
| T2 | ['28.231', '29.2905', '15.1315', '14.1782']
| T3 | ['23.9964', '5.97019', '40.2276', '11.0803']
Final MC results using: 1024 samples
Average: 143.38007941873053
Std: 9.33307443080061

